In [ ]:
import numpy as np
import math
from scipy.special import gammaincc


def is_negative(value):
    return value < 0

def is_zero(value):
    return value == 0

def cephes_lgam(x):
    return math.lgamma(x)

def cephes_igamc(a, x):
    # Complementary incomplete gamma function
    return math.gamma(a) - math.gamma(a, x)

def NonOverlappingTemplateMatchings(m,n,N,epsilon):
    numOfTemplates = [0, 0, 2, 4, 6, 12, 20, 40, 74, 148, 284, 568, 1116,
                      2232, 4424, 8848, 17622, 35244, 70340, 140680, 281076, 562152]

    N = N
    M = n // N
    print(M,N,n)
    try:
        Wj = np.zeros(N, dtype=int)
    except MemoryError:
        print("\tNONOVERLAPPING TEMPLATES TESTS ABORTED DUE TO INSUFFICIENT MEMORY")
        return

    lambda_value = (M - m + 1) / (2 ** m)
    varWj = M * (1.0 / (2.0 ** m) - (2.0 * m - 1.0) / (2.0 ** (2.0 * m)))
    directory = f"tamplate/template{m}"

    try:
        with open(directory, "r") as fp:
            sequence = np.zeros(m, dtype=int)


            if is_negative(lambda_value) or is_zero(lambda_value):
                print(f"\tLambda ({lambda_value}) is not positive!")
                return

            print("\t\t NONPERIODIC TEMPLATES TEST")
            print("-------------------------------------------------------------------------------------")
            print(f"\tLAMBDA = {lambda_value}\tM = {M}\tN = {N}\tm = {m}\tn = {n}")
            print("-------------------------------------------------------------------------------------")
            print("\t\tF R E Q U E N C Y")
            print("Template W_1 W_2 W_3 W_4 W_5 W_6 W_7 W_8 Chi^2 P_value Assignment Index")
            print("-------------------------------------------------------------------------------------")

            SKIP = 1 if numOfTemplates[m] < 200 else numOfTemplates[m] // 200
            numOfTemplates[m] //= SKIP

            sum_prob = 0.0
            pi = np.zeros(6)

            for i in range(2):
                pi[i] = math.exp(-lambda_value + i * math.log(lambda_value) - cephes_lgam(i + 1))
                sum_prob += pi[i]
            pi[0] = sum_prob

            for i in range(2, 6):
                pi[i - 1] = math.exp(-lambda_value + i * math.log(lambda_value) - cephes_lgam(i + 1))
                sum_prob += pi[i - 1]
            pi[5] = 1 - sum_prob

            for jj in range(min(200, numOfTemplates[m])):
                sum_val = 0
                bit_line=fp.readline().strip()
                for k in range(m):

                    sequence[k] = int(bit_line[k*2])

                print("sequence=",sequence)
                print(" ", end="")

                nu = np.zeros(6, dtype=int)

                for i in range(N):
                    W_obs = 0

                    for j in range(M - m + 1):
                        match = 1
                        for k in range(m):
                            if sequence[k] != epsilon[i * M + j + k]:
                                match = 0
                                break
                        if match == 1:
                            W_obs += 1
                            j += m - 1
                    Wj[i] = W_obs

                sum_val = 0
                chi2 = 0.0

                for i in range(N):
                    print(f"{Wj[i]:4d} ", end="")
                    chi2 += ((Wj[i] - lambda_value) / (varWj ** 0.5)) ** 2

                p_value =gammaincc(N / 2.0, chi2 / 2.0)

                if is_negative(p_value) or p_value > 1:
                    print("\t\tWARNING: P_VALUE IS OUT OF RANGE.")

                print(f"{chi2:9.6f} {p_value:.6f} {'FAILURE' if p_value < 0.01 else 'SUCCESS'} {jj:3d}")

                if SKIP > 1:
                    for _ in range((SKIP - 1) * 2 * m):
                        fp.readline()
    except FileNotFoundError:
        print(f"\tTemplate file {directory} does not exist")
    except MemoryError:
        print("\tInsufficient memory for required workspace.")

